In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive


'Agentic Ai.jpeg'		   preprocessed_keypoints
'Ai ML.jpg'			   preprocessed_keypoints_augmented
 attention_model_best.keras	   ransomware_dataset
 attention_model_final.keras	  'Siddhant Biswas (3).jpg'
 best_cnn_lstm_model.keras	   sign_model_checkpoints
 CNNLSTM.ipynb			   sign_recognition_dataset
'Colab Notebooks'		  'Sign Recognition Dataset'
 confusion_matrix.png		   sign_recognition_dataset_split
 fixed_videos			   split_keypoints
 gesture_dataset		   split_keypoints_v2
 gesture_dataset_raw		   thumbnail-1.jpeg
 gesture_dataset_ready.zip	   training_curves.png
 gesture_labels.npy		   training_log.csv
 gesture_model			   url_dataset_for_training
 gesture_model_cnn_lstm_v2.keras   url_detector_datasets
 Hackathon.jpeg			   url_malware_xgb_model.joblib
 important.gdoc			   urls_dataset_5000_balanced.csv
 label_encoder_48classes.pkl	   url_vectorizer.joblib
 label_encoder.pkl		   url_xgb_4classes_with_trust_final.model
 logs				   verification_logs
 malware_dataset		  'WhatsApp Image 

In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/urls_dataset_5000_balanced.csv")
df.head()


,url,label
0,https://www.learnonline.com?id=46941,0
1,https://www.learnonline.com/profile?id=8436,0
2,https://www.gov.in/login?id=22952,0
3,https://www.educationportal.org/blog?id=47638,0
4,https://www.gov.in/about,0


In [ ]:
df.head()
df.shape
df["label"].value_counts()


,count
label,
0,2500
1,2500


In [ ]:
import re
from urllib.parse import urlparse

def extract_url_features(url):
    parsed = urlparse(url)
    return {
        "url_length": len(url),
        "num_dots": url.count("."),
        "num_slashes": url.count("/"),
        "num_digits": sum(c.isdigit() for c in url),
        "has_ip": 1 if re.search(r"\d+\.\d+\.\d+\.\d+", url) else 0,
        "has_https": 1 if parsed.scheme == "https" else 0,
        "has_at": 1 if "@" in url else 0,
        "has_exe": 1 if ".exe" in url.lower() else 0,
        "has_zip": 1 if ".zip" in url.lower() else 0,
        "num_special_chars": sum(url.count(c) for c in "-_?=&%")
    }


In [ ]:
import pandas as pd

rows = []
for _, r in df.iterrows():
    f = extract_url_features(r["url"])
    f["label"] = r["label"]
    rows.append(f)

features_df = pd.DataFrame(rows)
features_df.head()


,url_length,num_dots,num_slashes,num_digits,has_ip,has_https,has_at,has_exe,has_zip,num_special_chars,label
0,36,2,2,5,0,1,0,0,0,2,0
1,43,2,3,4,0,1,0,0,0,2,0
2,33,2,3,5,0,1,0,0,0,2,0
3,45,2,3,5,0,1,0,0,0,2,0
4,24,2,3,0,0,1,0,0,0,0,0


In [ ]:
features_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   url_length         5000 non-null   int64
 1   num_dots           5000 non-null   int64
 2   num_slashes        5000 non-null   int64
 3   num_digits         5000 non-null   int64
 4   has_ip             5000 non-null   int64
 5   has_https          5000 non-null   int64
 6   has_at             5000 non-null   int64
 7   has_exe            5000 non-null   int64
 8   has_zip            5000 non-null   int64
 9   num_special_chars  5000 non-null   int64
 10  label              5000 non-null   int64
dtypes: int64(11)
memory usage: 429.8 KB


In [ ]:
from sklearn.model_selection import train_test_split

X = features_df.drop("label", axis=1)
y = features_df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)


In [ ]:
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier

n_trees = 200
batch_size = 10   # number of trees added per iteration

model = RandomForestClassifier(
    n_estimators=0,          # start with 0 trees
    warm_start=True,         # VERY IMPORTANT
    random_state=42,
    n_jobs=-1
)

for i in tqdm(range(0, n_trees, batch_size), desc="Training Random Forest"):
    model.n_estimators += batch_size
    model.fit(X_train_s, y_train)


Training Random Forest: 100%|██████████| 20/20 [00:01<00:00, 18.49it/s]


In [ ]:
len(model.estimators_)


200

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = model.predict(X_test_s)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 1.0

Confusion Matrix:
 [[491   0]
 [  0 509]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       491
           1       1.00      1.00      1.00       509

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000



In [ ]:
import joblib

joblib.dump(model, "/content/drive/MyDrive/url_malware_rf_model.pkl")
joblib.dump(scaler, "/content/drive/MyDrive/url_scaler.pkl")


['/content/drive/MyDrive/url_scaler.pkl']

In [ ]:
test_urls = [
    "https://www.google.com",
    "http://secure-update123.site/update.exe"
]

for url in test_urls:
    feats = extract_url_features(url)
    feats_df = pd.DataFrame([feats])
    feats_scaled = scaler.transform(feats_df)
    pred = model.predict(feats_scaled)[0]
    print(url, "→", "Malicious" if pred == 1 else "Benign")


https://www.google.com → Benign
http://secure-update123.site/update.exe → Malicious
